<img align="left" width="220" height="110" src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/USGS_logo_green.png">


<center>&nbsp;&nbsp;&nbsp;**SquamaterCreateEntitiesAndAttributesFromDataDictionaryV1-0**</center>

<center>&nbsp;&nbsp;&nbsp;Jupyter Notebook Used to create FGDC XML formatted Detailed Entity and Attribute Section from a data dictionary for insertion into G3 metadata</center>

<br><br>
Contact Phil Brown, https://www.usgs.gov/staff-profiles/philip-j-brown or visit Phil Brown's git hub "repo", https://github.com/pbrown-usgs?tab=repositories regarding any questions or concerns pertaining to this Jupyter notebook.

**V1.0 -** 
- Import a data dictionary CSV containing entity names, and entity and attribute definitions
- Parse names and definitions from imported data dictionary
- Open data CSV
- Corrrelate values in data dictionary with csv data
- Calculate Min and Max of data channels
- Create Detailed Metadata in FDGDC XML UTF 8 ASCII format for insertion into G3 metadata templates

<center> **Notes for future development** </center>
- It may be useful to perform checks on the metadat template being used before inserting EandA?
- It may be useful to insert the EandA directly into the metadata - version 1 relys on a user cut and paste?
- It may be useful to operate on files stored in sharepoint instead of the users harddrive?

### Instructions
- Create a template in UTF8 ASCII FGDC XML format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using this notebook.

#### To execute a function/command select a cell and Hold-Shift + Press-Enter


In [9]:
print ("All Systems Go - Jupyter Notebooks Is Working Correctly\n!!! PLEASE BE SURE TO LOAD LIBRARIES IN THE NEXT STEP !!!")

All Systems Go - Jupyter Notebooks Is Working Correctly
!!! PLEASE BE SURE TO LOAD LIBRARIES IN THE NEXT STEP !!!


In [23]:
# Load required Libraries
import sys
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
import datetime
import dateutil.parser
import time
import ssl
import json
import sys
import glob
#Create Function that allows one to implement markdown display when printing
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
print('Libraries Loaded')

Libraries Loaded


In [29]:
#Set paths to parent and children metadata
#The 'r' signifies a string literal. Be sure to use this when designating any paths.
#Examples below call from one's local hardrive.  
csvDataDictionaryPath = r"C:\CurrentWork\DataReleases\EastRiver2017" 
csvDataDictionaryName = "EastRiver2017_DataDictionary_ProcessedAEMData.csv"
csvDataPath = r"C:\CurrentWork\DataReleases\EastRiver2017" 
csvDataName = "EastRiver2017_ProcessedAEMData_100series.csv"
csvDataDictionaryPathName = csvDataDictionaryPath + "\\" + csvDataDictionaryName
csvDataPathName = csvDataPath + "\\" + csvDataName
#ParentMetadata = 'CaweloCA2016_flightlines.xml'
#ChildPath = r"C:\CurrentWork\DataReleases\SanJoaquinValley\Processed AEM data"
#ChildMetadata = 'CaweloCA2016_AEMProcessedData.xml'

# But we can also do this using Sharepoint Directories:

In [30]:
#Get a directory listing for the hell of it
#Note that this ends up being a multidimensional array with the last dimension containing the file names
csvDataDirList = next(os.walk(csvDataPath))[2]
csvDataDirList

['EastRiver2017_DataDictionary_EM_Mag.csv',
 'EastRiver2017_DataDictionary_ProcessedAEMData.csv',
 'EastRiver2017_DataDictionary_Rad.csv',
 'EastRiver2017_DataDictionary_ResistivityIPModels.csv.xlsx',
 'EastRiver2017_ProcessedAEMData_100series.csv',
 'EastRiver2017_ProcessedAEMData_200series.csv',
 'EastRiver2017_ProcessedAEMData_300series.csv',
 'EastRiver2017_ProcessedAEMData_400series.csv',
 'EastRiver2017_Radiometrics.csv',
 'EastRiver2017_ResistivityIPModels_100series.csv',
 'EastRiver2017_ResistivityIPModels_200series.csv',
 'EastRiver2017_ResistivityIPModels_300series.csv',
 'EastRiver2017_ResistivityIPModels_400series.csv']

In [31]:
# Or one can get a listing of the full path not parsed out using glob as seen below.  
# One can also filter by file type using wildcards such as *.*
csvDataDirListFullPathCulled = glob.glob(os.path.join(csvDataPath, '*.csv*'),  recursive=True)
csvDataDirListFullPathCulled

['C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_DataDictionary_EM_Mag.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_DataDictionary_ProcessedAEMData.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_DataDictionary_Rad.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_DataDictionary_ResistivityIPModels.csv.xlsx',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_ProcessedAEMData_100series.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_ProcessedAEMData_200series.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_ProcessedAEMData_300series.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_ProcessedAEMData_400series.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_Radiometrics.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRiver2017_ResistivityIPModels_100series.csv',
 'C:\\CurrentWork\\DataReleases\\EastRiver2017\\EastRive

In [32]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
#for more on pandas visit https://pandas.pydata.org/

datdicDataFrame = pd.read_csv(csvDataDictionaryPathName, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
datdicDataFrame

,Table,Field_Name,Definition,Units,Source
0,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",LINE,Line Number,LLLL,U.S. Geological Survey
1,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",TIMESTAMP,DateTime format,Decimal days,U.S. Geological Survey
2,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",E_UTM13N,"Easting, WGS84 Universal Transverse Mercator (...",Meters [m],U.S. Geological Survey
3,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",N_UTM13N,"Northing, WGS84 Universal Transverse Mercator ...",Meters [m],U.S. Geological Survey
4,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",ELEVATION,Digital Elevation Model,Meters [m],U.S. Geological Survey
5,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",ALT,"Altitude above ground surface, derived from pr...",Meters [m],U.S. Geological Survey
6,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",NUMDATA,Number of data points used in the inversion,NaN,U.S. Geological Survey
7,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",DATA[i],"Normalized, primary field corrected, processed...",volts/meters^4*Amps [V/(m4*A)],U.S. Geological Survey
8,"EastRiver2017_ProcessedAEMData_[100,200,300 or...",DATASTD[i],Standard deviation associated with DATA[i]; i ...,NaN,U.S. Geological Survey
9,EastRiver2017_AEMGateTime.csv,GATENUM,"index number for time gate i, from 0 to 51",NaN,U.S. Geological Survey


## Looks like we are missing the  EastRiver2017_AEMGateTime.csv file?

- I'll proceed like it doesn't exist and ignore those last two channels mentioned and that file when I create the metadata for now
- I'll copy some cells from before when I had this file - maybe it'l work with a little modification on your end?

In [33]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
#for more on pandas visit https://pandas.pydata.org/

csvDataFrame = pd.read_csv(csvDataPathName, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
csvDataFrame

,LINE,TIMESTAMP,E_UTM13N,N_UTM13N,ELEVATION,ALT,NUMDATA,DATA[0],DATA[1],DATA[2],...,DATASTD[42],DATASTD[43],DATASTD[44],DATASTD[45],DATASTD[46],DATASTD[47],DATASTD[48],DATASTD[49],DATASTD[50],DATASTD[51]
0,11000,43028.608507,318627.2,4301051.0,3230.3,33,43,5.899000e-12,5.060000e-12,4.373000e-12,...,0.2120,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
1,11000,43028.608519,318614.0,4301073.9,3227.0,36,43,6.329000e-12,5.342000e-12,4.579000e-12,...,0.2140,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
2,11000,43028.608530,318600.8,4301096.8,3226.4,35,43,6.156000e-12,5.157000e-12,4.423000e-12,...,0.2370,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
3,11000,43028.608542,318587.6,4301119.6,3226.4,33,43,5.794000e-12,4.832000e-12,4.170000e-12,...,0.2530,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
4,11000,43028.608553,318574.4,4301142.2,3223.2,35,43,5.726000e-12,4.807000e-12,4.160000e-12,...,0.2480,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
5,11000,43028.608565,318561.2,4301164.9,3220.7,37,43,5.768000e-12,4.908000e-12,4.284000e-12,...,0.2280,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
6,11000,43028.608576,318548.0,4301187.6,3219.4,38,43,5.831000e-12,4.967000e-12,4.385000e-12,...,0.2220,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
7,11000,43028.608588,318534.9,4301210.4,3215.8,40,43,5.996000e-12,5.030000e-12,4.377000e-12,...,0.2260,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
8,11000,43028.608600,318522.1,4301233.3,3211.8,43,43,6.177000e-12,5.203000e-12,4.444000e-12,...,0.2270,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000
9,11000,43028.608611,318509.3,4301256.3,3209.1,44,44,6.236000e-12,5.334000e-12,4.579000e-12,...,0.2170,0.3020,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000,-9999.0000


In [39]:
# now get an array of the header names and print them out.
csvHeaderList = list(csvDataFrame.columns.values)
sortedCsvHeaderList = sorted(csvHeaderList)
ddFieldNames = datdicDataFrame['Field_Name'].tolist()
sortedDdFieldNames = sorted(ddFieldNames)
print ('List of Names from Data:')
for i in range(len(sortedCsvHeaderList)):
 print (str(i) + ',' + sortedCsvHeaderList[i])

print ('\n\nList of Names from Data Dictionary:')

for i in range(len(sortedDdFieldNames)):
 print (str(i) + ',' + sortedDdFieldNames[i])

#for i in range(0,13):
 #print (str(i) + ',' + sortedCsvHeaderList[i] + '::' + sortedDdFieldNames[i])

##for i in range(145,154):
 ##print (str(i) + ',' + sortedCsvHeaderList[i] + '::' + sortedDdFieldNames[158-i])
##for i in range(17,25):
 ##print (str(i) + ',' + sortedCsvHeaderList[i+130] + '::' + sortedDdFieldNames[i])

List of Names from Data Dictionary:
0,ALT
1,DATASTD[0]
2,DATASTD[10]
3,DATASTD[11]
4,DATASTD[12]
5,DATASTD[13]
6,DATASTD[14]
7,DATASTD[15]
8,DATASTD[16]
9,DATASTD[17]
10,DATASTD[18]
11,DATASTD[19]
12,DATASTD[1]
13,DATASTD[20]
14,DATASTD[21]
15,DATASTD[22]
16,DATASTD[23]
17,DATASTD[24]
18,DATASTD[25]
19,DATASTD[26]
20,DATASTD[27]
21,DATASTD[28]
22,DATASTD[29]
23,DATASTD[2]
24,DATASTD[30]
25,DATASTD[31]
26,DATASTD[32]
27,DATASTD[33]
28,DATASTD[34]
29,DATASTD[35]
30,DATASTD[36]
31,DATASTD[37]
32,DATASTD[38]
33,DATASTD[39]
34,DATASTD[3]
35,DATASTD[40]
36,DATASTD[41]
37,DATASTD[42]
38,DATASTD[43]
39,DATASTD[44]
40,DATASTD[45]
41,DATASTD[46]
42,DATASTD[47]
43,DATASTD[48]
44,DATASTD[49]
45,DATASTD[4]
46,DATASTD[50]
47,DATASTD[51]
48,DATASTD[5]
49,DATASTD[6]
50,DATASTD[7]
51,DATASTD[8]
52,DATASTD[9]
53,DATA[0]
54,DATA[10]
55,DATA[11]
56,DATA[12]
57,DATA[13]
58,DATA[14]
59,DATA[15]
60,DATA[16]
61,DATA[17]
62,DATA[18]
63,DATA[19]
64,DATA[1]
65,DATA[20]
66,DATA[21]
67,DATA[22]
68,DATA[23]
69,DATA

In [40]:
#Creat Min and Max Arrays
#Calculate Min Max and print result:
csvMax = []
csvMin = []

#Loop through arrays and callculate values
for i in range(len(csvHeaderList)):
 print (csvHeaderList[i] + ': ' + str(i))
 strCol = csvHeaderList[i]
 csvMax.append(csvDataFrame[strCol].max())
 csvMin.append(csvDataFrame[strCol].min())
 print ('Max. Value ' + csvHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + csvHeaderList[i] + ' :' + str(csvMin[i]))

LINE: 0
Max. Value LINE :12630
Min. Value LINE :11000
TIMESTAMP: 1
Max. Value TIMESTAMP :43047.9741435185
Min. Value TIMESTAMP :43027.619895833304
E_UTM13N: 2
Max. Value E_UTM13N :343083.7
Min. Value E_UTM13N :317320.2
N_UTM13N: 3
Max. Value N_UTM13N :4325872.8
Min. Value N_UTM13N :4292520.8
ELEVATION: 4
Max. Value ELEVATION :3673.1
Min. Value ELEVATION :0.0
ALT: 5
Max. Value ALT :209
Min. Value ALT :19
NUMDATA: 6
Max. Value NUMDATA :52
Min. Value NUMDATA :20
DATA[0]: 7
Max. Value DATA[0] :9.411e-10
Min. Value DATA[0] :1.826e-12
DATA[1]: 8
Max. Value DATA[1] :9.032000000000001e-10
Min. Value DATA[1] :1.8650000000000002e-12
DATA[2]: 9
Max. Value DATA[2] :8.699e-10
Min. Value DATA[2] :1.727e-12
DATA[3]: 10
Max. Value DATA[3] :8.412000000000001e-10
Min. Value DATA[3] :1.6219999999999998e-12
DATA[4]: 11
Max. Value DATA[4] :8.057999999999999e-10
Min. Value DATA[4] :1.5630000000000001e-12
DATA[5]: 12
Max. Value DATA[5] :7.657e-10
Min. Value DATA[5] :1.5090000000000001e-12
DATA[6]: 13
Max. Va

In [41]:
#Create lists from Panadas rows
ddDefinitions = datdicDataFrame['Definition'].tolist()
ddSource = datdicDataFrame['Source'].tolist()
ddUnit = datdicDataFrame['Units'].tolist()
entAttiAll = ''
   
#Search lists and print out matching fields as well as indexed min and max to EandA
for i in range(len(csvHeaderList)):
 for j in range (len(ddFieldNames)):
    if csvHeaderList[i][:8] == ddFieldNames[j][:8]: # [:8] means just compare the first 8 chars of any string - solves matching problem with indexing the data dictionary to the actual data
     entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + str(csvHeaderList[i]) + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + str(ddDefinitions[j]) + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>' + str(ddSource[j]) + '</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + str(ddUnit[j]) + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
     entAttiAll = entAttiAll + entAtti
print (str(entAttiAll))


				<attr>
					<attrlabl>LINE</attrlabl>
					<attrdef>Line Number</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>11000</rdommin>
							<rdommax>12630</rdommax>
							<attrunit>LLLL</attrunit>
						</rdom>
					</attrdomv>
				</attr>
				<attr>
					<attrlabl>TIMESTAMP</attrlabl>
					<attrdef>DateTime format</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>43027.619895833304</rdommin>
							<rdommax>43047.9741435185</rdommax>
							<attrunit>Decimal days</attrunit>
						</rdom>
					</attrdomv>
				</attr>
				<attr>
					<attrlabl>E_UTM13N</attrlabl>
					<attrdef>Easting, WGS84 Universal Transverse Mercator (UTM) zone 13 north</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>317320.2</rdommin>
							<rdommax>343083.7</rdommax>
							<attrunit>Meters [m]</attrunit>
						</rdom>
					</attrdomv>
				</attr>
	

## Now lets do this for the second CSV file, EastRiver2017_AEMGateTime.csv?
- the cells below were from before when we had two files.
- note that we read in a different file and panda's data frame - these variables are not designated in this notebook yet so the blow cells will not work.

In [66]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
#for more on pandas visit https://pandas.pydata.org/

csvDataFrame2 = pd.read_csv(csvFilePath2, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
csvDataFrame2

,GATENUM,SEGMENT,CENTERTIME,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,0.0,1.0,0.000033,NaN,NaN,NaN,NaN
1,1.0,1.0,0.000042,NaN,NaN,NaN,NaN
2,2.0,1.0,0.000054,NaN,NaN,NaN,NaN
3,3.0,1.0,0.000068,NaN,NaN,NaN,NaN
4,4.0,1.0,0.000086,NaN,NaN,NaN,NaN
5,5.0,1.0,0.000109,NaN,NaN,NaN,NaN
6,6.0,1.0,0.000137,NaN,NaN,NaN,NaN
7,7.0,1.0,0.000172,NaN,NaN,NaN,NaN
8,8.0,1.0,0.000218,NaN,NaN,NaN,NaN
9,9.0,1.0,0.000275,NaN,NaN,NaN,NaN


In [67]:
# now get an array of the header names and print them out.
csvHeaderList2 = list(csvDataFrame2.columns.values)
sortedCsvHeaderList2 = sorted(csvHeaderList2)
ddFieldNames = datdicDataFrame['Field_Name'].tolist()
sortedDdFieldNames = sorted(ddFieldNames)

for i in range(len(sortedCsvHeaderList2)):
 print (str(i) + ',' + sortedCsvHeaderList2[i])
for i in range(len(sortedDdFieldNames)):
 print (str(i) + ',' + sortedDdFieldNames[i])

for i in range(0,13):
 print (str(i) + ',' + sortedCsvHeaderList2[i] + '::' + sortedDdFieldNames[i])

##for i in range(145,154):
 ##print (str(i) + ',' + sortedCsvHeaderList[i] + '::' + sortedDdFieldNames[158-i])
##for i in range(17,25):
 ##print (str(i) + ',' + sortedCsvHeaderList[i+130] + '::' + sortedDdFieldNames[i])

0,CENTERTIME
1,GATENUM
2,SEGMENT
3,Unnamed: 3
4,Unnamed: 4
5,Unnamed: 5
6,Unnamed: 6
0,ALT
1,CENTERTIME
2,DATASTD_i
3,DATA_dBdt_i
4,DEM
5,GATENUM
6,LINE
7,NUMDATA
8,SEGMENT
9,TIME
10,X_UTM12N
11,Y_UTM12N
0,CENTERTIME::ALT
1,GATENUM::CENTERTIME
2,SEGMENT::DATASTD_i
3,Unnamed: 3::DATA_dBdt_i
4,Unnamed: 4::DEM
5,Unnamed: 5::GATENUM
6,Unnamed: 6::LINE


IndexError: list index out of range

In [69]:
#Creat Min and Max Arrays
#Calculate Min Max and print result:
csvMax = []
csvMin = []

#Loop through arrays and callculate values
for i in range(len(csvHeaderList2)):
 print (csvHeaderList2[i] + ': ' + str(i))
 strCol = csvHeaderList2[i]
 csvMax.append(csvDataFrame2[strCol].max())
 csvMin.append(csvDataFrame2[strCol].min())
 print ('Max. Value ' + csvHeaderList2[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + csvHeaderList2[i] + ' :' + str(csvMin[i]))

GATENUM: 0
Max. Value GATENUM :42.0
Min. Value GATENUM :0.0
SEGMENT: 1
Max. Value SEGMENT :2.0
Min. Value SEGMENT :1.0
CENTERTIME: 2
Max. Value CENTERTIME :0.0107
Min. Value CENTERTIME :3.3100000000000005e-05
Unnamed: 3: 3
Max. Value Unnamed: 3 :nan
Min. Value Unnamed: 3 :nan
Unnamed: 4: 4
Max. Value Unnamed: 4 :nan
Min. Value Unnamed: 4 :nan
Unnamed: 5: 5
Max. Value Unnamed: 5 :nan
Min. Value Unnamed: 5 :nan
Unnamed: 6: 6
Max. Value Unnamed: 6 :nan
Min. Value Unnamed: 6 :nan


In [68]:
#Create lists from Panadas rows
ddDefinitions = datdicDataFrame['Definition'].tolist()
ddSource = datdicDataFrame['Source'].tolist()
ddUnit = datdicDataFrame['Units'].tolist()
entAttiAll = ''
   
#Search lists and print out matching fields as well as indexed min and max to EandA
for i in range(len(csvHeaderList2)):
 for j in range (len(ddFieldNames)):
    if csvHeaderList[i][:8] == ddFieldNames[j][:8]: # [:8] means just compare the first 8 chars of any string - solves matching problem with indexing the data dictionary to the actual data
     entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + str(csvHeaderList2[i]) + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + str(ddDefinitions[j]) + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>' + str(ddSource[j]) + '</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + str(ddUnit[j]) + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
     entAttiAll = entAttiAll + entAtti
print (str(entAttiAll))


				<attr>
					<attrlabl>GATENUM</attrlabl>
					<attrdef>Line Number</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>500101</rdommin>
							<rdommax>600401</rdommax>
							<attrunit>LLLLLL</attrunit>
						</rdom>
					</attrdomv>
				</attr>
				<attr>
					<attrlabl>Unnamed: 3</attrlabl>
					<attrdef>Date and time of measurement</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>42651.8130787037</rdommin>
							<rdommax>42651.9786689815</rdommax>
							<attrunit>Decimal days</attrunit>
						</rdom>
					</attrdomv>
				</attr>
				<attr>
					<attrlabl>Unnamed: 5</attrlabl>
					<attrdef>Land surface elevation derived from USGS National Elevation Dataset, 1 arc second resolution, referenced to the North American Vertical Datum of 1988 (NAVD88)</attrdef>
					<attrdefs>U.S. Geological Survey</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>126.9</rdommin>
						